In [245]:
import importlib
import labs
import preds
import cnnm
import inv 

importlib.reload(labs)
importlib.reload(preds)
importlib.reload(cnnm)
importlib.reload(inv)

from labs import *
from inv import *
from preds import *
from cnnm import *

# Predictor
- 훈련 데이터에 오라벨된 것이 있는지

In [3]:
image_paths = grep_files('/data/ephemeral/home/dataset/docsy-deskew/', exts=['jpg'])
len(image_paths)

0it [00:00, ?it/s]

3140

In [4]:
processor = AutoImageProcessor.from_pretrained('facebook/convnextv2-large-22k-384')

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [182]:
ckpt_path = './ckpt/exp-convnext-large-384-easy-rotator-dice-loss-add-last_epoch.ckpt'
classes_path = '/data/ephemeral/home/dataset/dtc/doc_classes.json'
predictor = Predictor(CNN, processor, ckpt_path, classes_path)

Some weights of ConvNextV2ForImageClassification were not initialized from the model checkpoint at facebook/convnextv2-large-22k-384 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([17]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1536]) in the checkpoint and torch.Size([17, 1536]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# TEST

In [55]:
def run(idx):
    p = image_paths[idx]
    df = predictor.feed_bty(p)

    cols_to_drop = df.columns[(df < 0.01).all(axis=0)]
    filtered_df = df.drop(columns=cols_to_drop)
    
    display(filtered_df)
    show_img(path=p)

In [118]:
# df = run(64)

# TEST ALL

In [141]:
items = predictor.test(image_paths[:])

  0%|          | 0/3140 [00:00<?, ?it/s]

In [142]:
write_list('./cnv2-easy-rot-dice-loss.txt', items)

# Compare

In [234]:
wrongs = read_list_from_txt('inconsist_list.txt')
base_dir = '/data/ephemeral/home/dataset/docsy-deskew/'
paths = [os.path.join(base_dir, i) for i in wrongs]

In [241]:
answers = [(item[0][:-6] + '.jpg', np.argmax(np.array(item[1])), np.array(item[1]).max()) for item in items]
df = pd.DataFrame(answers, columns=['ID', 'target', 'prob'])
hists = [ df ]

In [246]:
inv = ImageNavigator(paths, hists, id2kor)